## iPyLeaflet
[ipyleaflet](https://github.com/ellisonbg/ipyleaflet) is a bridge between jupyter notebooks and the [leaflet](http://leafletjs.com/)  javascript library for drawing maps.

ipyleaflet comes with a few examples notebooks (this notebook was derived from one) but very little documentation,
for more documentation read the [Leaflet IPA](http://leafletjs.com/reference.html)

For installation directions, see the README on [ipyleaflet](https://github.com/ellisonbg/ipyleaflet)

[GeoJson](https://tools.ietf.org/html/rfc7946#section-3.1.7) is a good way to add data on top of an ipyleaflet map.

For other backgrounds than open streets, you can alter the ipyleaflet.py file to point to one of [these map providers](https://leaflet-extras.github.io/leaflet-providers/preview/)

In [4]:
from ipyleaflet import (
    Map,
    Marker,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    GeoJSON,
    DrawControl
)

## Visualizing the distribution of the observations

## Load the required libraries

In [5]:
import pandas as pd
import numpy as np
import sklearn as sk
import urllib
import math
import pylab as plt

import sys
sys.path.append('./lib')

from leaflet import *

In [11]:
data_dir = "../../Data/Weather/"
from pickle import load
with open(data_dir+'/PRCP_residuals_PCA.pickle','rb') as file:
    Res_eig=load(file)
print Res_eig.keys()

#stations.pkl
with open(data_dir+'/stations.pkl','rb') as file:
    Stations=load(file)
Stations.head()


['eigen-vecs', 'stations']


,latitude,longitude,elevation,state,name,GSNFLAG,HCNFLAG,WMOID
ACW00011604,17.1167,-61.7833,10.1,NaN,ST JOHNS COOLIDGE FLD,NaN,NaN,NaN
ACW00011647,17.1333,-61.7833,19.2,NaN,ST JOHNS,NaN,NaN,NaN
AE000041196,25.3330,55.5170,34.0,NaN,SHARJAH INTER. AIRP,GSN,NaN,41196.0
AF000040930,35.3170,69.0170,3366.0,NaN,NORTH-SALANG,GSN,NaN,40930.0
AG000060390,36.7167,3.2500,24.0,NaN,ALGER-DAR EL BEIDA,GSN,NaN,60390.0


In [13]:
import pandas as pd
Eig=pd.DataFrame(Res_eig['eigen-vecs'].transpose())
Eig.head()

,0,1,2,3,4,5,6,7,8,9
0,-0.185341,-0.020449,-0.008295,-0.054145,0.198246,0.010514,-0.021180,0.019655,-0.035092,0.051325
1,-0.220957,-0.000550,-0.037611,0.035276,-0.020148,0.031355,-0.004738,0.027568,-0.030141,0.042644
2,-0.196401,-0.013453,-0.022541,-0.046610,0.172506,0.026251,0.017254,0.064244,-0.033889,0.043561
3,-0.220014,-0.044693,-0.024703,0.084321,0.042965,0.075924,-0.036930,-0.124126,-0.074709,0.139002
4,-0.218817,-0.021760,-0.032854,0.064147,0.011672,0.042747,-0.006191,-0.096269,-0.012654,0.124704


In [14]:
Eig['station']=Res_eig['stations']
Eig=Eig.set_index('station')
print Res_eig['stations'][:5]
Eig.head()

[u'USC00420487', u'USS0011H25S', u'USC00428885', u'USC00428828', u'USS0011H55S']


,0,1,2,3,4,5,6,7,8,9
station,,,,,,,,,,
USC00420487,-0.185341,-0.020449,-0.008295,-0.054145,0.198246,0.010514,-0.021180,0.019655,-0.035092,0.051325
USS0011H25S,-0.220957,-0.000550,-0.037611,0.035276,-0.020148,0.031355,-0.004738,0.027568,-0.030141,0.042644
USC00428885,-0.196401,-0.013453,-0.022541,-0.046610,0.172506,0.026251,0.017254,0.064244,-0.033889,0.043561
USC00428828,-0.220014,-0.044693,-0.024703,0.084321,0.042965,0.075924,-0.036930,-0.124126,-0.074709,0.139002
USS0011H55S,-0.218817,-0.021760,-0.032854,0.064147,0.011672,0.042747,-0.006191,-0.096269,-0.012654,0.124704


In [15]:
table=Eig.join(Stations,how='left')

In [16]:
table=table[['name','latitude','longitude','elevation',0,1,2,3,4,5,6,7]]
table.head(4)

,name,latitude,longitude,elevation,0,1,2,3,4,5,6,7
station,,,,,,,,,,,,
USC00420487,BEAR LAKE SP,41.9583,-111.3922,1806.9,-0.185341,-0.020449,-0.008295,-0.054145,0.198246,0.010514,-0.021180,0.019655
USS0011H25S,LITTLE BEAR,41.4000,-111.8167,1996.4,-0.220957,-0.000550,-0.037611,0.035276,-0.020148,0.031355,-0.004738,0.027568
USC00428885,UINTAH,41.1500,-111.9167,1473.1,-0.196401,-0.013453,-0.022541,-0.046610,0.172506,0.026251,0.017254,0.064244
USC00428828,TRENTON,41.9153,-111.9128,1357.9,-0.220014,-0.044693,-0.024703,0.084321,0.042965,0.075924,-0.036930,-0.124126


In [17]:
import pylab as plt
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']

## Map

In [20]:
min_lat,max_lat,min_long,max_long = box = (40.9139, 42.5872, -111.0456, -112.1489)
center = [(min_lat+max_lat)/2, (min_long+max_long)/2]
zoom = 9
m = Map(default_tiles=TileLayer(opacity=0.6), center=center, zoom=zoom)
r = Rectangle(bounds=[[min_lat,min_long],[max_lat,max_long]], weight=5, fill_opacity=0.0)
m += r

lat_margin=(max_lat-min_lat)/4
long_margin=(max_long-min_long)/4
for index,row in table.iterrows():
    _lat=row['latitude']
    _long=row['longitude']
    for col in range(4):
        _coef=row[col]
        if np.isnan(_coef):
            continue
        r=abs(_coef)/5
        #print r
        color=colors[col]
        signs=[[+1,+1],[+1,-1],[-1,-1],[-1,+1]]
        lat_sign,long_sign=signs[col]
        triangle=[(_lat,_long),(_lat+lat_sign*r,_long),(_lat,_long+long_sign*r),(_lat,_long)]
        if _coef<0:
            poly= Polygon(locations=triangle, weight=0,
                    color=color, opacity=0, fill_opacity=0.7,
                    fill_color=color)
        else:
            poly= Polygon(locations=triangle, weight=2,
                    color=color, opacity=0.8, fill_opacity=0,
                    fill_color=color)
        m +=poly   
m    

### excercises:
* Add a legend that relates the colors to values.
* Leaflet supports a variety of maps. See if you can get a topographical map as the background.

In [10]:
%matplotlib inline
pdf.plot.scatter(x='elevation',y='avg(coeff_1)');

NameError: name 'pdf' is not defined